In [1]:
from tsai.models import MINIROCKET
import data_loader 
from tsai.data.all import *
import pandas as pd
import numpy as np


In [2]:
data_hri = data_loader.DataLoader_HRI("../data/")

46091
47790
31987
73387
38286
33089
36595
50689
40690
62681
35599
88581
42999
43177
48292
38399
50791
37598
51697
148204
59803
78200
103402
80580
38287
41590
36387
30591
53790
34191
83191
102187
51097
37006
46600
56592
66196
79606
64693
41693
31486
43392
50694
37178
35790
34692
40496
60102
47999
51102
52687
44392
71209
127960
54191
56193
140983
42892
59592
45295
44394
50893
128409
81193
62990
104624
39051
148182
52898
105181
126602


Number of sessions (data) parsed: 55 16
Number of sessions (labels) parsed: 55 16


Number of rows in merged dataframes: 2528752 790266 3151940 1153106


In [3]:
data_hri.train_X

,frame,timestamp_openface,session,AU01_r_openface,AU01_c_openface,AU02_r_openface,AU04_r_openface,AU05_r_openface,AU06_r_openface,AU07_r_openface,...,logRelF0-H1-A3_sma3nz_opensmile,F1frequency_sma3nz_opensmile,F1bandwidth_sma3nz_opensmile,F1amplitudeLogRelF0_sma3nz_opensmile,F2frequency_sma3nz_opensmile,F2bandwidth_sma3nz_opensmile,F2amplitudeLogRelF0_sma3nz_opensmile,F3frequency_sma3nz_opensmile,F3bandwidth_sma3nz_opensmile,F3amplitudeLogRelF0_sma3nz_opensmile
0,1,0.000,0,0.06,0.0,0.0,0.0,0.0,0.00,0.27,...,-11.261256,1336.56570,801.48070,4.584261,2376.2742,777.46844,6.609108,3251.1199,803.74780,8.525012
1,1,0.000,0,0.06,0.0,0.0,0.0,0.0,0.00,0.27,...,-12.525172,1053.50590,885.96160,-63.545944,2090.7010,866.97430,-61.715313,2989.5070,853.38380,-60.595320
2,1,0.000,0,0.06,0.0,0.0,0.0,0.0,0.00,0.27,...,0.000000,745.35150,902.49396,-131.676150,1787.0573,890.26660,-130.039730,2701.9048,860.84320,-129.715650
3,1,0.000,0,0.06,0.0,0.0,0.0,0.0,0.00,0.27,...,0.000000,707.16046,872.19946,-201.000000,1758.6289,868.56230,-201.000000,2681.4700,860.75640,-201.000000
4,2,0.033,0,0.14,0.0,0.0,0.0,0.0,0.00,0.22,...,0.000000,1017.59814,803.31195,-201.000000,2057.5974,805.89680,-201.000000,2913.1592,821.95140,-201.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52599,15780,530.387,9,0.00,0.0,0.0,0.0,0.0,1.41,0.87,...,2.916806,393.79288,1000.48627,0.000000,1411.7754,958.08600,0.466851,2680.6570,813.98390,-3.859359
52600,15780,530.387,9,0.00,0.0,0.0,0.0,0.0,1.41,0.87,...,1.056943,409.32385,942.38470,0.000000,1431.6973,911.73750,4.743028,2699.5020,799.54755,-2.155445
52601,15781,530.420,9,0.00,0.0,0.0,0.0,0.0,1.43,0.80,...,-1.192079,405.62488,907.16770,0.000000,1417.8939,893.27277,6.111391,2606.0671,860.86570,0.308648
52602,15781,530.420,9,0.00,0.0,0.0,0.0,0.0,1.43,0.80,...,-12.533787,454.16330,810.28630,0.000000,1481.1710,944.38007,16.271430,2587.0542,703.42334,15.169826


In [4]:
# print lenght of each each session in the dataset (per dataframe)

train_Y_counts = data_hri.train_Y["session"].value_counts().sort_index()
train_X_counts = data_hri.train_X["session"].value_counts().sort_index()
val_Y_counts = data_hri.val_Y["session"].value_counts().sort_index()
val_X_counts = data_hri.val_X["session"].value_counts().sort_index()

# print side by side
print("Train")
print(pd.concat([train_Y_counts, train_X_counts], axis=1))
print("Val")
print(pd.concat([val_Y_counts, val_X_counts], axis=1))


Train
          count  count
session               
0         46091  46103
1         51697  51702
10        31987  31132
11        38286  37708
12        36595  36344
13        40690  40487
14        35599  35605
15        42999  43004
16        48292  48298
17        38399  38402
18        50791  50800
19        37598  37602
2         83191  83200
20        59803  29903
21        78200  39101
22       103402  51702
23        80580  40298
24        38287  37670
25        41590  40992
26        36387  35650
27        30591  30278
28        53790  53807
29        34191  34199
3         50694  50700
30        51097  51102
31        37006  37007
32        46600  46607
33        56592  56604
34        66196  33098
35        79606  39804
36        64693  64706
37        41693  41704
38        31486  30836
39        43392  43098
4         54191  54199
40        35790  35227
41        34692  34188
42        40496  40502
43        60102  60104
44        47999  48003
45        51102  51101
46   

In [5]:
# print session 9 of train_Y
print(data_hri.train_Y[data_hri.train_Y["session"] == "9"].head(52604))

       frame session  Duration - ss.msec  Begin Time - ss.msec  \
0          1       9                0.01                  0.00   
1          2       9                0.01                  0.01   
2          3       9                0.01                  0.02   
3          4       9                0.01                  0.03   
4          5       9                0.01                  0.04   
...      ...     ...                 ...                   ...   
52599  52600       9                0.01                  0.06   
52600  52601       9                0.01                  0.07   
52601  52602       9                0.01                  0.08   
52602  52603       9                0.01                  0.09   
52603  52604       9                0.01                  0.10   

       UserAwkwardness  RobotMistake  InteractionRupture  
0                    0             0                   0  
1                    0             0                   0  
2                    0        

In [6]:
tr = data_hri.train_X
# print all columns with NaN values
print(tr.columns[tr.isna().any()].tolist())
# print all columns without NaN values
print(tr.columns[~tr.isna().any()].tolist())
# print number of nan per column
for col in tr.columns:
    print(f"{col}: {tr[col].isna().sum()}")

[' timestamp_openface', ' AU01_r_openface', ' AU01_c_openface', ' AU02_r_openface', ' AU04_r_openface', ' AU05_r_openface', ' AU06_r_openface', ' AU07_r_openface', ' AU09_r_openface', ' AU10_r_openface', ' AU12_r_openface', ' AU14_r_openface', ' AU15_r_openface', ' AU17_r_openface', ' AU20_r_openface', ' AU23_r_openface', ' AU25_r_openface', ' AU26_r_openface', ' AU45_r_openface', ' AU02_c_openface', ' AU04_c_openface', ' AU05_c_openface', ' AU06_c_openface', ' AU07_c_openface', ' AU09_c_openface', ' AU10_c_openface', ' AU12_c_openface', ' AU14_c_openface', ' AU15_c_openface', ' AU17_c_openface', ' AU20_c_openface', ' AU23_c_openface', ' AU25_c_openface', ' AU26_c_openface', ' AU28_c_openface', ' AU45_c_openface', 'frame_id_openpose', 'vel_1_x_openpose', 'vel_1_y_openpose', 'vel_8_x_openpose', 'vel_8_y_openpose', 'dist_1_8_openpose', 'vel_dist_1_8_openpose', 'dist_7_0_openpose', 'dist_4_0_openpose', 'vel_7_x_openpose', 'vel_7_y_openpose', 'vel_4_x_openpose', 'vel_4_y_openpose', 'vel_di

In [7]:
# print rows where timestamp_openface is NaN
print(tr[tr[" timestamp_openface"].isna()])

       frame   timestamp_openface session   AU01_r_openface   AU01_c_openface  \
35604  10682                  NaN      14               NaN               NaN   
37004  11102                  NaN      31               NaN               NaN   
37005  11102                  NaN      31               NaN               NaN   
37006  11102                  NaN      31               NaN               NaN   
46604  13982                  NaN      32               NaN               NaN   
46605  13982                  NaN      32               NaN               NaN   
46606  13982                  NaN      32               NaN               NaN   

        AU02_r_openface   AU04_r_openface   AU05_r_openface   AU06_r_openface  \
35604               NaN               NaN               NaN               NaN   
37004               NaN               NaN               NaN               NaN   
37005               NaN               NaN               NaN               NaN   
37006               NaN    

In [8]:
val_X_TS, val_Y_TS, train_X_TS, train_Y_TS = data_hri.get_timeseries_form(
        intervallength=100, stride=100)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)